## RLDMUU 2025
#### Backward Induction
jakub.tluczek@unine.ch

Today your task would be to implement the backwards induction algorithm for the following MDP (you can also find it in `src/MDP/MDP.py` on our github):

In [24]:
import numpy as np

## This a discrete MDP with a finite number of states and actions
class DiscreteMDP:
    ## initalise a random MDP with
    ## n_states: the number of states
    ## n_actions: the number of actions
    ## Optional arguments:
    ## P: the state-action-state transition matrix so that P[s,a,s_next] is the probability of s_next given the current state-action pair (s,a)
    ## R: The state-action reward matrix so that R[s,a] is the reward for taking action a in state s.
    def __init__(self, n_states, n_actions, P = None, R = None):
        self.n_states = n_states # the number of states of the MDP
        self.n_actions = n_actions # the number of actions of the MDP
        if (P is None):
            self.P = np.zeros([n_states, n_actions, n_states]) # the transition probability matrix of the MDP so that P[s,a,s'] is the probabiltiy of going to s' from (s,a)
            for s in range(self.n_states):
                for a in range(self.n_actions):
                    self.P[s,a] = np.random.dirichlet(np.ones(n_states)) # generalisation of Beta to multiple outcome
        else:
            self.P = P
        if (R is None):
            self.R = np.zeros([n_states, n_actions]) # the expected reward for each action and state
            # generate uniformly random transitions and 0.1 bernoulli rewards
            for s in range(self.n_states):
                for a in range(self.n_actions):
                    self.R[s,a] = np.round(np.random.uniform(), decimals=1)
        else:
            self.R = R
        
        # check transitions
        for s in range(self.n_states):
            for a in range(self.n_actions):
                #print(s,a, ":", self.P[s,a,:])
                assert(abs(np.sum(self.P[s,a,:])-1) <= 1e-3)
                assert((self.P[s,a,:] <= 1).all())
                assert((self.P[s,a,:] >= 0).all())
                
    # get the probability of next state j given current state s, action a, i.e. P(j|s,a)
    def get_transition_probability(self, state, action, next_state):
        return self.P[state, action, next_state]
    
    # get the vector of probabilities over next states P( . | s,a)
    def get_transition_probabilities(self, state, action):
        return self.P[state, action]
    
    # Get the reward for the current state action.
    # It can also be interpreted as the expected reward for the state and action.
    def get_reward(self, state, action):
        return self.R[state, action]

### Backward induction

As a reminder, in the backward induction algorithm we consider an MDP with finite horizon $T$, and for each step of the algorithm, we compute:

$$ V_t(s) = max_{a \in A} \left[ R(s,a) + \sum_{s' \in S} P(s' | s,a)V_{t+1}(s') \right]$$

where $R(s,a)$ is a reward received by picking action $a$ in state $s$, $P(s'|s,a)$ is the probability of transitioning into next state $s'$, and $V_{t+1}(s')$ is the value of said next state at time $t+1$. We can also say, that for the last timestep (with index $T-1$) the next state value is 0 for every state $V_T(s) = 0$. Consecutively, the action $a$ which maximizes $V_t(s)$ can be described as policy $\pi_t(s)$ at state $s$ and time $t$, 

Your task is to implement this algorithm. Remember to do the inverse iteration, and iterate from $T-1$ to $0$, not the other way around. Your function should return matrix of state values for each $s$ and $t$, as well as resulting policy:

In [28]:
# TODO: Implement backwards induction
def backwards_induction(mdp, T):
    V = np.zeros((mdp.n_states, T+1))
    Q = np.zeros((mdp.n_states, mdp.n_actions, T+1))
    policy = np.zeros((mdp.n_states, T+1))

    for t in range(T-1, -1, -1):
        for s in range(mdp.n_states):
            for a in range(mdp.n_actions):
                Q[s,a,t] = mdp.get_reward(s,a) + np.sum([mdp.get_transition_probability(s,a,sp) * V[sp, t+1] for sp in range(mdp.n_states)])
            V[s,t] = np.max(Q[s, :, t])
            policy[s,t] = np.argmax(Q[s, :, t])
    
    return V[:,:-1], policy[:,:-1]

In [30]:
STATES = 5
ACTIONS = 3

T = 15

mdp = DiscreteMDP(STATES, ACTIONS)

V, policy = backwards_induction(mdp, T)

print(V)

print(policy)


[[12.55479557 11.70739238 10.85999265 10.01260101  9.16522833  8.31789996
   7.4706752   6.62369196  5.77726895  4.9321344   4.08992803  3.25414681
   2.43042892  1.6189997   0.8       ]
 [12.61329919 11.76589235 10.91848406 10.0710724   9.22365281  8.37621471
   7.5287333   6.68115087  5.83333359  4.98497559  4.13539696  3.28312609
   2.42508736  1.55741897  0.7       ]
 [12.43458147 11.5871762  10.73977158  9.8923685   9.04496901  8.19757797
   7.35020668  6.50288169  5.65566571  4.80871095  3.96237696  3.11734433
   2.27491295  1.4460592   0.7       ]
 [12.41643174 11.56902561 10.72161897  9.87421117  9.02680065  8.17938383
   7.33195251  6.48448854  5.63695467  4.7892831   3.94138043  3.09347808
   2.24976446  1.43496047  0.7       ]
 [12.90981566 12.06240972 11.21500354 10.36759681  9.52018879  8.67277772
   7.82535952  6.9779245   6.13044967  5.28287853  4.43506673  3.58666662
   2.73694182  1.88279752  1.        ]]
[[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1.